In [1]:
import sys
print(sys.version, sys.platform, sys.executable)

3.7.6 (default, Aug  9 2020, 21:13:30) 
[Clang 11.0.3 (clang-1103.0.32.62)] darwin /Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/bin/python3.7


In [2]:
import os
os.chdir('/Users/layne/Desktop/pydatasci')
os.getcwd()

'/Users/layne/Desktop/pydatasci'

In [3]:
import h5py
import pydatasci as pds
from pydatasci import aidb
import os, sqlite3, io, gzip 
import pandas as pd
import numpy as np
import pyarrow
from pyarrow import csv as pc

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

import keras
from keras import metrics
from keras.models import Sequential, load_model
from keras.callbacks import History
from keras.layers import Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.constraints import maxnorm
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
from importlib import reload; aidb.delete_db(True); reload(aidb); aidb.create_db()


=> Success - deleted database file at path:
/Users/layne/Library/Application Support/pydatasci/aidb.sqlite3


=> Success - created database file for machine learning metrics at path:
/Users/layne/Library/Application Support/pydatasci/aidb.sqlite3


=> Success - created the following tables within database:
['algorithm', 'batch', 'dataset', 'featureset', 'fold', 'foldset', 'hyperparamcombo', 'hyperparamset', 'job', 'label', 'preprocess', 'result', 'splitset']



---

In [5]:
dataset = aidb.Dataset.from_file(
	path = 'data/iris.tsv' 
	,file_format = 'tsv'
	,name = 'tab-separated plants'
	,perform_gzip = True
    ,dtype = 'float64'
)

In [6]:
label_name = 'target'

In [7]:
label = dataset.make_label(columns=[label_name])

In [8]:
featureset = dataset.make_featureset(exclude_columns=[label_name])

In [9]:
splitset = featureset.make_splitset(
	label_id = label.id
	, size_test = 0.20
	, size_validation = 0.12
)

In [10]:
#foldset = splitset.make_foldset(fold_count=6)

# aidb

### Define Preprocess (Optional)

Going to have to make sure that I am uniformly importing the packages. 
`from sklearn.preprocessing import *`

In [11]:
encoder_features = StandardScaler()

In [12]:
encoder_labels = OneHotEncoder(sparse=False)

In [13]:
#params_encode_labels = {"sparse": [False]}

### Define Algorithm

In [14]:
def function_model_build(**hyperparameters):
    model = Sequential()
    model.add(Dense(9, input_shape=(4,), activation='relu', kernel_initializer='he_uniform', name='fc1')) # first hidden layer
    model.add(Dense(hyperparameters['l2_neuron_count'], activation='relu', kernel_initializer='he_uniform', name='fc2'))
    model.add(Dense(3, activation='softmax', name='output'))

    model.compile(optimizer=hyperparameters['optimizer'], loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [15]:
def function_model_train(model, samples_train, samples_evaluate, **hyperparameters):
    model.fit(
        samples_train["features"]
        , samples_train["labels"]
        , validation_data = (
            samples_evaluate["features"]
            , samples_evaluate["labels"]
        )
        , verbose = 0
        , batch_size = 3
        , epochs = hyperparameters['epochs']
        , callbacks=[History()]
    )
    return model

In [16]:
def function_model_evaluate(model, samples_evaluate, **hyperparameters):
    results = model.evaluate(samples_evaluate["features"], samples_evaluate["labels"], verbose=0)
    #print("Loss = " + str(results[0]) + " // Accuracy = " + str(results[1]) )
    return results

In [17]:
# hyperparameters = {
#     "l1_neuron_count": [9, 18]
#     , "l2_neuron_count": [9, 18]
#     , "optimizer": ["adamax", "adam"]
#     , "epochs": [30, 60, 90]
#     , "batch_size": [3, 5]
# }

In [18]:
hyperparameters = {
    "l2_neuron_count": [9, 18]
    , "optimizer": ["adamax", "adam"]
    , "epochs": [10, 20]
}

### Stage the Experiment and Parameters

In [19]:
algorithm = aidb.Algorithm.create(
    library = "Keras"
    , description = "dense, 2 layers, medium height"
	, function_model_build = function_model_build
	, function_model_train = function_model_train
	, function_model_evaluate = function_model_evaluate
)

In [20]:
preprocess = aidb.Preprocess.from_splitset(
    splitset_id = splitset.id
    , description = "standard scaling on features"
    , encoder_features = encoder_features
    , encoder_labels = encoder_labels
)

In [21]:
hyperparamset = aidb.Hyperparamset.from_algorithm(
    algorithm_id = algorithm.id
    , preprocess_id = preprocess.id
    , description = "experimenting with number of epochs"
	, hyperparameters = hyperparameters
)

In [22]:
hyperparamset.hyperparamcombo_count

8

In [23]:
batch = aidb.Batch.from_algorithm(
    algorithm_id = algorithm.id
    , splitset_id = splitset.id
    , hyperparamset_id = hyperparamset.id
    , foldset_id = None #foldset.id
    , only_folded_training = False
)

In [24]:
batch.job_count

8

In [25]:
batch.run_jobs(verbose=False)

🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 8/8 [00:10<00:00,  1.30s/it]


In [26]:
batch.get_statuses()

{1: 'Queued',
 2: 'Queued',
 3: 'Queued',
 4: 'Queued',
 5: 'Queued',
 6: 'Queued',
 7: 'Queued',
 8: 'Queued'}

In [27]:
#batch.stop_jobs()

In [29]:
batch.jobs[0].results[0].get_model()

In [30]:
batch.jobs[0].results[0].evaluations

{'train': [0.8290887475013733, 0.5686274766921997],
 'validation': [0.8197234272956848, 0.6111111044883728],
 'test': [0.8392254710197449, 0.5666666626930237]}

In [31]:
batch.jobs[0].results[0].model_history

{'loss': [1.1361279487609863,
  1.0814762115478516,
  1.0396240949630737,
  1.0023505687713623,
  0.9693678021430969,
  0.9382761120796204,
  0.9110653400421143,
  0.8860802054405212,
  0.8626300096511841,
  0.8403457999229431],
 'accuracy': [0.343137264251709,
  0.343137264251709,
  0.3921568691730499,
  0.4215686321258545,
  0.44117647409439087,
  0.45098039507865906,
  0.47058823704719543,
  0.5,
  0.5098039507865906,
  0.5490196347236633],
 'val_loss': [1.0702990293502808,
  1.0278077125549316,
  0.9908008575439453,
  0.957991898059845,
  0.9270286560058594,
  0.9018315076828003,
  0.8793914318084717,
  0.8583993911743164,
  0.8385547995567322,
  0.8197234272956848],
 'val_accuracy': [0.3333333432674408,
  0.3333333432674408,
  0.5,
  0.5,
  0.5,
  0.5555555820465088,
  0.5555555820465088,
  0.5555555820465088,
  0.6111111044883728,
  0.6111111044883728]}

---

---

---

### Data to use

In [11]:
samples = splitset.to_numpy()

In [12]:
train_features = samples["train"]["features"]
validation_features = samples["validation"]["features"]
test_features = samples["test"]["features"]

In [13]:
train_labels = samples["train"]["labels"]
validation_labels = samples["validation"]["labels"]
test_labels = samples["test"]["labels"]

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

---

In [14]:
folds = [None]
fset = ['a','b','c']
folds = folds + fset

On jobs need a `Job.from_hyperparamset()`

---

^ Alternatively, pass in a dictionary of values for that keyword.

^ Or just pass the params directly into the encoder function if you don't want to hypertune on them.

In [ ]:
encoder_labels = encoder.set_params(**encode_features_kwargs)

In [ ]:
encoder_labels_trained = encoder_labels.fit(train_labels)

In [ ]:
train_labels = encoder_labels_trained.transform(train_labels)
validation_labels = encoder_labels_trained.transform(validation_labels)
test_labels = encoder_labels_trained.transform(test_labels)

Create the preprocess object.

In [ ]:
preprocess = aidb.Preprocess.create(
    encode_labels_function = encoder
)

In [ ]:
feature_encoder = preprocess.preprocess_labels_function.set_params(**preprocess_features_kwargs)

In [ ]:
encoder_labels_trained = feature_encoder.fit(train_labels)

In [ ]:
train_labels = encoder_labels_trained.transform(train_labels)
validation_labels = encoder_labels_trained.transform(validation_labels)
test_labels = encoder_labels_trained.transform(test_labels)

^^^ Now do the same, but with kwargs stored in the hyperparamset attribute.

---

---

# Normal

ToDo - Does Pandas labels probably wants a series too, not a full dataframe? `.as_series()` method?

In [30]:
samples = splitset.to_numpy()

In [31]:
train_features = samples["train"]["features"]
test_features = samples["test"]["features"]

In [32]:
train_labels = samples["train"]["labels"]
test_labels = samples["test"]["labels"]

In [33]:
encoder = OneHotEncoder(sparse=False)
train_labels = encoder.fit_transform(train_labels)
test_labels = encoder.fit_transform(test_labels)

In [34]:
model = Sequential()
model.add(Dense(13, input_shape=(4,), activation='relu', kernel_initializer='he_uniform', name='fc1')) # first hidden layer
model.add(Dense(3, activation='softmax', name='output'))
model.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 13)                65        
_________________________________________________________________
output (Dense)               (None, 3)                 42        
Total params: 107
Trainable params: 107
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
# Train
def train():
    model.fit(
        train_features
        , train_labels
        , validation_data = (test_features, test_labels)
        , verbose = 0
        , batch_size = 6
        , epochs = 60
        , callbacks=[History()]
    )
    return model

In [37]:
model = train()

In [39]:
model.history.history

{'loss': [1.716982126235962,
  1.6168975830078125,
  1.5186580419540405,
  1.4281631708145142,
  1.3363103866577148,
  1.2430486679077148,
  1.1638587713241577,
  1.0838406085968018,
  1.0114461183547974,
  0.9434520602226257,
  0.883625328540802,
  0.8264437913894653,
  0.7812291979789734,
  0.7397616505622864,
  0.7080036997795105,
  0.6793625950813293,
  0.653673529624939,
  0.6317193508148193,
  0.6112250685691833,
  0.594214141368866,
  0.5785435438156128,
  0.5647279620170593,
  0.5523313879966736,
  0.5398412346839905,
  0.5295905470848083,
  0.519515872001648,
  0.5111671090126038,
  0.5023576617240906,
  0.4946964979171753,
  0.4876684248447418,
  0.48149698972702026,
  0.4745713472366333,
  0.4696151912212372,
  0.4632842540740967,
  0.4580068290233612,
  0.4531033933162689,
  0.4486708641052246,
  0.443932443857193,
  0.43989092111587524,
  0.4355388283729553,
  0.43247413635253906,
  0.4281100928783417,
  0.4246746897697449,
  0.4213531017303467,
  0.41772744059562683,
  0.

In [ ]:
model.weights

In [ ]:
with open('/trainHistoryDict', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
type(model)

In [ ]:
# Train
dir(model)

In [ ]:
plt.subplot(211)
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

plt.subplot(212)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()

plt.subplots_adjust(top=2, right=2)

plt.show()

---